### LLM interaction paper
Running fulltexts through chatgpt

In [1]:
import os

In [3]:
%cd c:\\Users\\Parv\\Doc\\Nexus_project

c:\Users\Parv\Doc\Nexus_project


c:\Users\Parv\anaconda3\envs\pytorch_gpu\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [5]:
with open('backend\openai_api_key.txt', 'r') as f:
    openai_key = f.readlines()[0]

os.environ['OPENAI_API_KEY'] = openai_key

<>:1: SyntaxWarning: invalid escape sequence '\o'
<>:1: SyntaxWarning: invalid escape sequence '\o'
C:\Users\Parv\AppData\Local\Temp\ipykernel_31412\1461466291.py:1: SyntaxWarning: invalid escape sequence '\o'
  with open('backend\openai_api_key.txt', 'r') as f:


In [6]:
from backend.backend_function import search_for_papers, retrieve_paper, bulk_retrieve_papers, get_paper_pdf_urls
from backend.text_extraction import extract_pdf_text_from_url

2025-09-05 16:28:26 INFO: Downloaded file to C:\Users\Parv\stanza_resources\resources.json
2025-09-05 16:28:26 WARNING: Language en package default expects mwt, which has been added
2025-09-05 16:28:26 INFO: Downloading these customized packages for language: en (English)...
| Processor       | Package           |
---------------------------------------
| tokenize        | combined          |
| mwt             | combined          |
| pos             | combined_charlm   |
| lemma           | combined_nocharlm |
| depparse        | combined_charlm   |
| pretrain        | conll17           |
| backward_charlm | 1billion          |
| forward_charlm  | 1billion          |

2025-09-05 16:28:26 INFO: File exists: C:\Users\Parv\stanza_resources\en\tokenize\combined.pt
2025-09-05 16:28:26 INFO: File exists: C:\Users\Parv\stanza_resources\en\mwt\combined.pt
2025-09-05 16:28:26 INFO: File exists: C:\Users\Parv\stanza_resources\en\pos\combined_charlm.pt
2025-09-05 16:28:26 INFO: File exists: C:\Us

2025-09-05 16:28:27 INFO: Downloaded file to C:\Users\Parv\stanza_resources\resources.json
2025-09-05 16:28:27 WARNING: Language en package default expects mwt, which has been added
2025-09-05 16:28:27 INFO: Loading these models for language: en (English):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| depparse  | combined_charlm   |

2025-09-05 16:28:27 INFO: Using device: cuda
2025-09-05 16:28:27 INFO: Loading: tokenize
2025-09-05 16:28:28 INFO: Loading: mwt
2025-09-05 16:28:28 INFO: Loading: pos
2025-09-05 16:28:31 INFO: Loading: lemma
2025-09-05 16:28:32 INFO: Loading: depparse
2025-09-05 16:28:33 INFO: Done loading processors!


### Getting test paper case

In [7]:
paper = search_for_papers('Motif caller')

ERROR:root:Retrieving bulk papers returned status 429 for 20 papers
ERROR:root:Retrieving bulk papers returned status 429 for 20 papers
ERROR:root:Retrieving bulk papers returned status 429 for 20 papers
ERROR:root:Retrieving bulk papers returned status 429 for 20 papers


In [8]:
paper = retrieve_paper(paper[0]['paperId'])

In [10]:
refs = bulk_retrieve_papers([i['paperId'] for i in paper['references']])

In [11]:
t = [paper] + refs

In [12]:
pdf_urls = get_paper_pdf_urls([paper] + refs)

ERROR:root:Error in extracting pdf url for {'paperId': '8289cb7b8806e4d4b345dc8fd02ff1f52fc4047f', 'externalIds': {'MAG': '1471723175', 'DBLP': 'conf/osdi/BalakrishnanBDEGHLOPR14', 'CorpusId': 417688}, 'corpusId': 417688, 'title': 'Pelican: A Building Block for Exascale Cold Data Storage', 'year': 2014, 'referenceCount': 19, 'citationCount': 82, 'influentialCitationCount': 14, 'openAccessPdf': {'url': '', 'status': None, 'license': None}, 'fieldsOfStudy': ['Computer Science'], 's2FieldsOfStudy': [{'category': 'Computer Science', 'source': 'external'}, {'category': 'Computer Science', 'source': 's2-fos-model'}, {'category': 'Engineering', 'source': 's2-fos-model'}], 'tldr': {'model': 'tldr@v2.0.0', 'text': 'Pelican performs well for cold workloads, providing high throughput with acceptable latency, and is compared against a traditional resource overprovisioned storage rack using a cross-validated simulator.'}, 'journal': {'pages': '351-365'}, 'authors': [{'authorId': '3074541', 'name': 

### Paper segmentation

In [20]:
import re

def segment_paper(text):
    headers = ["abstract", "introduction", "methods?", "results?", "discussion", 
               "conclusion", "references?"]
    # Put (?i) ONCE at the start for case-insensitive matching
    pattern = r"(?i)\b(" + "|".join(headers) + r")\b"
    sections = re.split(pattern, text)
    return [s.strip() for s in sections if s.strip()]



In [13]:
fulltexts = [extract_pdf_text_from_url(i) if i else None for i in pdf_urls]

Pdf extraction error HTTPSConnectionPool(host='advanced.onlinelibrary.wiley.com', port=443): Max retries exceeded with url: /doi/10.1002/advs.202411354 (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1000)')))


In [18]:
import regex as re
import requests
import fitz

In [15]:
pdf_url = pdf_urls[5]

In [16]:
pdf_url

'https://www.nature.com/articles/s41598-024-58386-z.pdf'

In [19]:
response = requests.get(pdf_url)
with open("temp.pdf", "wb") as f:
    f.write(response.content)

doc = fitz.open("temp.pdf")

In [20]:
doc.metadata

{'format': 'PDF 1.4',
 'title': 'Efficient DNA-based data storage using shortmer combinatorial encoding',
 'author': 'Inbal Preuss ',
 'subject': 'Scientific Reports, https://doi.org/10.1038/s41598-024-58386-z',
 'keywords': '',
 'creator': 'Springer',
 'producer': 'Adobe PDF Library 15.0; modified using iText® 5.3.5 ©2000-2012 1T3XT BVBA (SPRINGER SBM; licensed version)',
 'creationDate': "D:20240402072217+05'30'",
 'modDate': "D:20240902151110+02'00'",
 'trapped': '',
 'encryption': None}

In [50]:
def segment_paper(doc, toc):

    def define_match_locations_per_page(match_locations):
        """
        Based on matched section headings in the text (page numbers..), get the best estimate on where
        the text begins and ends
        """

        match_locs = []  # Final list

        for ptr, loc in enumerate(match_locations):
                
            if len(loc) >= 1:
                if len(loc) < 2:
                    best_match_loc = loc[0]

                else:
                    if ptr == 0: # First section on the page
                        best_match_loc = loc[0]
                    else:
                        # Select the last match as long as it is before the next section found
                        best_match_loc = len(loc) - 1
                        while True:
                            if ptr < len(match_locations) - 2:
                                next_match = match_locations[ptr + 1]
                                if len(next_match) > 0:
                                    if next_match[0] < loc[best_match_loc]:
                                        best_match_loc -= 1
                                    else:
                                        break
                                else:
                                    ptr += 1
                                    continue
                            else:
                                break
            else:
                best_match_loc = -1
            
            match_locs.append(best_match_loc)

        return match_locs

    per_page_dicts = []

    for page_num, page in enumerate(doc):

        text = page.get_text('text')

        section_headings_in_page = [
            i[1] for i in toc if i[2] == page_num + 1]  # Getting section headings from toc

        match_locations = []
        for ptr, section_heading in enumerate(section_headings_in_page):  # Getting all match locations
            text_matches = [
                    m.start() for m in re.finditer(section_heading, text, re.IGNORECASE)
                ]
            match_locations.append(text_matches)
        per_page_dict = {}
        final_match_locations = define_match_locations_per_page(match_locations)  # Final match locations for all section headings found
        for i in range(len(section_headings_in_page)):
            heading = section_headings_in_page[i]
            if i == 0:
                per_page_dict['prev'] = text[:final_match_locations[0]]
            if i == len(section_headings_in_page) - 1:
                per_page_dict[heading] = text[final_match_locations[i]:]
            else:
                per_page_dict[heading] = text[final_match_locations[i]: final_match_locations[i+1]]

        per_page_dicts.append(per_page_dict)
        
    segmented_paper = {}
    last_section = ""
    held_key = ""
    for i, j in enumerate(per_page_dicts):
        for key, value in j.items():
            if key == 'prev' and i > 0:
                segmented_paper[last_section] += value
            else:
                segmented_paper[key] = value

            held_key = key
        last_section = held_key 

    return segmented_paper



In [51]:

from tqdm import tqdm

segmented_papers = []
for pdf_url in tqdm(pdf_urls):
    try:
        response = requests.get(pdf_url)
        with open("temp.pdf", "wb") as f:
            f.write(response.content)
        doc = fitz.open("temp.pdf")
        toc = doc.get_toc(simple=True)
        print(toc)
    except Exception as e:
        print("pdf extraction error")
    seg_paper = segment_paper(doc, toc)
    segmented_papers.append(seg_paper)

  5%|▌         | 1/20 [00:00<00:02,  7.82it/s]

[[1, 'Introduction', 1], [1, 'Motif-Based Synthesis', 2], [1, 'Basecalling & Motif Search - The Baseline', 3], [2, 'Zero-error (ZE) Search', 3], [2, 'Approximate-matching (AM) Search', 3], [1, 'Motif Caller', 4], [2, 'Training Procedure', 4], [2, 'Decoding', 5], [1, 'Dataset', 5], [1, 'Results', 6], [2, 'Motif accuracies per read', 6], [2, 'Information recovered per block with increasing reads', 7], [2, 'Decoding percentage for diluted reads', 7], [2, 'Generalization of performance to alternative sequencing runs', 8], [2, 'Finetuning and Quality Proportions', 8], [1, 'Discussion', 9], [1, 'References', 9]]


 15%|█▌        | 3/20 [00:00<00:03,  4.86it/s]

[]
[[1, 'Introduction', 1], [1, 'Motif-Based DNA Storage', 2], [2, 'General Principle', 2], [2, 'Experimental Data Set', 2], [2, 'Channel Simulator', 3], [1, 'Coupon Collector Channels', 3], [2, 'Capacity of the CC(n,k,R) Channel', 4], [2, 'Erasure Version of the Coupon Collector Channel', 5], [2, 'Coupon Collector Channel with Interference', 6], [2, 'Read-Write Cost Trade-Off', 7], [1, 'Non-Binary LDPC Codes Over the Coupon Collector Channels', 8], [2, 'No Interference', 8], [2, 'Interference', 8], [2, 'Protograph-Based Non-Binary SC-LDPC Codes', 9], [2, 'Combinatorial Explosion in Decoding Complexity', 9], [1, 'Numerical Results', 10], [1, 'Conclusion', 10], [1, 'References', 11]]


 20%|██        | 4/20 [00:00<00:03,  5.27it/s]

...

[[1, 'Abstract', 1], [2, 'Background', 1], [2, 'Results', 1], [2, 'Conclusions', 1], [2, 'Keywords', 1], [1, 'Background', 1], [1, 'Methods', 2], [2, 'The semi-global DP algorithm', 2], [2, 'Naïve difference recurrence relations', 3], [2, 'The proposed algorithm', 4], [2, 'Relation to other DP algorithms', 5], [2, 'Library implementation', 5], [3, 'Finding the cell with the maximum value', 5], [3, 'Traceback flags and generating alignment path strings', 7], [3, 'Computation of score profile vectors and prefetching sequences', 8], [3, 'Miscellaneous implementation techniques', 8], [4, 'Register usage:', 8], [4, 'Branch predictor consideration:', 9], [1, 'Results', 9], [2, 'Comparison to existing implementations', 11], [1, 'Discussion', 13], [1, 'Conclusions', 14], [1, 'Additional files', 14], [2, 'Additional file 1', 14], [2, 'Additional file 2', 14], [1, 'Abbreviations', 14], [1, 'Acknowledgments', 14], [1, 'Funding', 14], [1, 'Availability of data and materials', 14], [1, 'About this 

 75%|███████▌  | 15/20 [00:15<00:03,  1.37it/s]

[]


100%|██████████| 20/20 [00:16<00:00,  1.23it/s]

[[1, 'Not Found', 1]]
pdf extraction error
pdf extraction error
pdf extraction error
pdf extraction error


In [42]:
segmented_papers

[]

In [38]:
seg_paper

{'prev': '1\nVol.:(0123456789)\nScientific Reports |         (2024) 14:7731  \n| https://doi.org/10.1038/s41598-024-58386-z\nwww.nature.com/scientificreports\nEfficient DNA‑based data storage \nusing shortmer combinatorial \nencoding\nInbal\xa0Preuss\u200a1,3*, Michael\xa0Rosenberg\u200a2, Zohar\xa0Yakhini\u200a1,3\xa0& Leon\xa0Anavy\u200a1,3\nData storage in DNA has recently emerged as a promising archival solution,\xa0offering space-efficient \nand long-lasting digital storage solutions. Recent studies suggest leveraging the inherent redundancy \nof synthesis and sequencing technologies by using composite DNA alphabets. A major challenge \nof this approach involves the noisy inference process, obstructing\xa0large composite alphabets. This \npaper introduces a novel approach for DNA-based data storage, offering, in some implementations, \na 6.5-fold increase in logical density over standard DNA-based storage systems, with near-zero \nreconstruction error. Combinatorial DNA encoding u

In [29]:
per_page_dicts

[{'prev': '1\nVol.:(0123456789)\nScientific Reports |         (2024) 14:7731  \n| https://doi.org/10.1038/s41598-024-58386-z\nwww.nature.com/scientificreports\nEfficient DNA‑based data storage \nusing shortmer combinatorial \nencoding\nInbal\xa0Preuss\u200a1,3*, Michael\xa0Rosenberg\u200a2, Zohar\xa0Yakhini\u200a1,3\xa0& Leon\xa0Anavy\u200a1,3\nData storage in DNA has recently emerged as a promising archival solution,\xa0offering space-efficient \nand long-lasting digital storage solutions. Recent studies suggest leveraging the inherent redundancy \nof synthesis and sequencing technologies by using composite DNA alphabets. A major challenge \nof this approach involves the noisy inference process, obstructing\xa0large composite alphabets. This \npaper introduces a novel approach for DNA-based data storage, offering, in some implementations, \na 6.5-fold increase in logical density over standard DNA-based storage systems, with near-zero \nreconstruction error. Combinatorial DNA encoding 

In [76]:
text

'11\ncapacity limits that are outside of the reach of the earlier\nschemes. We show how to calculate those fundamental limits\nand how to reason about the associated trade-off between the\nread and write costs. We devise an error-correcting scheme\nthat approaches these limits in practice and show how to\ncircumvent the combinatorial complexity explosion in our\ndecoding algorithm at the cost of some additional redundancy.\nOverall, the paper offers a powerful framework to reason about\nand incorporate error correcting coding in one of the most\npromising approaches to DNA data storage.\nREFERENCES\n[1] L. Ceze, J. Nivala, and K. Strauss, “Molecular digital data storage using\nDNA,” Nature Reviews Genetics, vol. 20, no. 8, pp. 456–466, May 2019.\n[2] Y. Yan, N. Pinnamaneni, S. Chalapati, C. Crosbie, and R. Appuswamy,\n“Scaling logical density of DNA storage with enzymatically-ligated\ncomposite motifs,” Scientific Reports, vol. 13, no. 1, p. 15978, Sep.\n2023.\n[3] I. Preuss, M. Rosenb

In [ ]:
toc[0][2]

[1, 'Introduction', 1]

In [38]:
block_dict = {}
page_num = 1
for page in doc: # Iterate all pages in the document
    file_dict = page.get_text('html') # Get the page dictionary
    #block = file_dict['blocks'] # Get the block information
    #block_dict[page_num] = block # Store in block dictionary
    #page_num += 1 # Increase the page value by 1

    break

In [39]:
with open('page_doc.txt', 'w') as f:
    f.write(file_dict)

In [69]:

split_page = file_dict.split('\n')

for line in split_page:

    if line.startswith('<p'):
        #print(line)

        pattern = re.compile(
            r'font-family:([^;,]+).*?font-size:([\d.]+)pt.*?>(.*?)<',
            re.DOTALL
        )
        matches = pattern.findall(line)

        for font_family, font_size, text in matches:
            print("Font family:", font_family.strip(),
                "Font size:", font_size,
                "Text:", text)

Font family: NimbusRomNo9L Font size: 7.0 Text: 1
Font family: NimbusRomNo9L Font size: 23.9 Text: Coding Over Coupon Collector Channels
Font family: NimbusRomNo9L Font size: 23.9 Text: for Combinatorial Motif-Based DNA Storage
Font family: NimbusRomNo9L Font size: 11.0 Text: Roman Sokolovskii, Parv Agarwal, Luis Alberto Croquevielle, Zijian Zhou, and Thomas Heinis
Font family: NimbusRomNo9L Font size: 9.0 Text: Abstract
Font family: NimbusRomNo9L Font size: 9.0 Text: &#x2014;Encoding
Font family: NimbusRomNo9L Font size: 9.0 Text: information
Font family: NimbusRomNo9L Font size: 9.0 Text: in
Font family: NimbusRomNo9L Font size: 9.0 Text: combinations
Font family: NimbusRomNo9L Font size: 9.0 Text: of
Font family: NimbusRomNo9L Font size: 9.0 Text: pre-
Font family: NimbusRomNo9L Font size: 9.0 Text: synthesised deoxyribonucleic acid (DNA) strands (referred to as
Font family: NimbusRomNo9L Font size: 9.0 Text: motifs
Font family: NimbusRomNo9L Font size: 9.0 Text: ) is an interesting

In [108]:
!pip install unidecode

   ---------------------------------------- 0.0/235.8 kB ? eta -:--:--
   - -------------------------------------- 10.2/235.8 kB ? eta -:--:--
   ----- --------------------------------- 30.7/235.8 kB 640.0 kB/s eta 0:00:01
   --------------- ------------------------ 92.2/235.8 kB 1.0 MB/s eta 0:00:01
   ---------------------------------------- 235.8/235.8 kB 1.6 MB/s eta 0:00:00


DEPRECATION: Loading egg at c:\users\parv\anaconda3\envs\pytorch_gpu\lib\site-packages\ont_bonito-0.8.1-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [109]:
from unidecode import unidecode

output = []
for page in doc:
    output += page.get_text("blocks")
previous_block_id = 0 # Set a variable to mark the block id
for block in output:
    if block[6] == 0: # We only take the text
        if previous_block_id != block[5]: # Compare the block number
            print("\n")
        plain_text = unidecode(block[4])
        print(plain_text)

1



Coding Over Coupon Collector Channels
for Combinatorial Motif-Based DNA Storage



Roman Sokolovskii, Parv Agarwal, Luis Alberto Croquevielle, Zijian Zhou, and Thomas Heinis



Abstract--Encoding
information
in
combinations
of
pre-
synthesised deoxyribonucleic acid (DNA) strands (referred to as
motifs) is an interesting approach to DNA storage that could
potentially circumvent the prohibitive costs of nucleotide-by-
nucleotide DNA synthesis. Based on our analysis of an empirical
data set from HelixWorks, we propose two channel models for
this setup (with and without interference) and analyse their
fundamental limits. We propose a coding scheme that approaches
those limits by leveraging all information available at the output
of the channel, in contrast to earlier schemes developed for
a similar setup by Preuss et al. We highlight an important
connection between channel capacity curves and the fundamental
trade-off between synthesis (writing) and sequencing (reading)
costs, and off

In [110]:

import pandas as pd

import re
spans = pd.DataFrame(columns=['xmin', 'ymin', 'xmax', 'ymax', 'text', 'tag'])
rows = []
for page_num, blocks in block_dict.items():
    for block in blocks:
        if block['type'] == 0:
            for line in block['lines']:
                for span in line['spans']:
                    xmin, ymin, xmax, ymax = list(span['bbox'])
                    font_size = span['size']
                    text = unidecode(span['text'])
                    span_font = span['font']
                    is_upper = False
                    is_bold = False
                    if "bold" in span_font.lower():
                        is_bold = True
                    if re.sub("[\(\[].*?[\)\]]", "", text).isupper():
                        is_upper = True
                    if text.replace(" ","") !=  "":
                        rows.append((xmin, ymin, xmax, ymax, text,                              is_upper, is_bold, span_font, font_size))
                        span_df = pd.DataFrame(rows, columns=['xmin','ymin','xmax','ymax', 'text', 'is_upper','is_bold','span_font', 'font_size'])

<>:19: SyntaxWarning: invalid escape sequence '\('
<>:19: SyntaxWarning: invalid escape sequence '\('
C:\Users\Parv\AppData\Local\Temp\ipykernel_27480\2512976593.py:19: SyntaxWarning: invalid escape sequence '\('
  if re.sub("[\(\[].*?[\)\]]", "", text).isupper():


In [116]:
span_df.loc[span_df['is_upper'] == True]

,xmin,ymin,xmax,ymax,text,is_upper,is_bold,span_font,font_size
31,135.619019,369.739471,150.722321,379.702087,I. I,True,False,NimbusRomNo9L-Regu,9.962600
32,151.221024,371.250580,213.371811,379.220673,NTRODUCTION,True,False,NimbusRomNo9L-Regu,7.970100
174,209.111603,72.000130,216.001755,80.021255,A,True,False,CMMI7,8.021126
177,209.111603,60.589195,216.001755,68.610321,A,True,False,CMMI7,8.021126
196,329.744781,72.000130,336.634918,80.021255,A,True,False,CMMI7,8.021126
...,...,...,...,...,...,...,...,...,...
4448,521.108093,739.905334,527.484192,747.875488,R,True,False,CMMI8,7.970100
4466,146.598022,189.884598,153.243073,199.847198,R,True,False,NimbusRomNo9L-Regu,9.962600
4467,153.741028,191.395691,202.398483,199.365784,EFERENCES,True,False,NimbusRomNo9L-Regu,7.970100
4469,67.223030,215.156677,89.180649,223.126770,"DNA,""",True,False,NimbusRomNo9L-Regu,7.970100


In [111]:
spans

,xmin,ymin,xmax,ymax,text,tag


In [91]:
t.search('Motif-Based DNA Storage')

[Quad(Point(266.5853576660156, 86.52171325683594), Point(528.1879272460938, 86.52171325683594), Point(266.5853576660156, 110.43201446533203), Point(528.1879272460938, 110.43201446533203)),
 Quad(Point(122.23931884765625, 599.9053344726562), Point(231.31983947753906, 599.9053344726562), Point(122.23931884765625, 609.867919921875), Point(231.31983947753906, 609.867919921875)),
 Quad(Point(311.97796630859375, 594.8651733398438), Point(419.82305908203125, 594.8651733398438), Point(311.97796630859375, 604.8277587890625), Point(419.82305908203125, 604.8277587890625))]

In [80]:
page = doc.load_page(0)

In [84]:
print(page.get_text("html", sort=True))

<div id="page0" style="width:612.0pt;height:792.0pt">
<p style="top:25.6pt;left:559.5pt;line-height:7.0pt"><span style="font-family:NimbusRomNo9L,serif;font-size:7.0pt;color:#000000">1</span></p>
<p style="top:57.6pt;left:103.5pt;line-height:23.9pt"><span style="font-family:NimbusRomNo9L,serif;font-size:23.9pt;color:#000000">Coding Over Coupon Collector Channels</span></p>
<p style="top:85.5pt;left:83.8pt;line-height:23.9pt"><span style="font-family:NimbusRomNo9L,serif;font-size:23.9pt;color:#000000">for Combinatorial Motif-Based DNA Storage</span></p>
<p style="top:119.2pt;left:87.7pt;line-height:11.0pt"><span style="font-family:NimbusRomNo9L,serif;font-size:11.0pt;color:#000000">Roman Sokolovskii, Parv Agarwal, Luis Alberto Croquevielle, Zijian Zhou, and Thomas Heinis</span></p>
<p style="top:176.8pt;left:58.9pt;line-height:9.0pt"><b><i><span style="font-family:NimbusRomNo9L,serif;font-size:9.0pt;color:#000000">Abstract</span></i></b><b><span style="font-family:NimbusRomNo9L,serif;fo

In [66]:
dir(doc)

['FontInfos',
 'FormFonts',
 'Graftmaps',
 'InsertedImages',
 'ShownPages',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '_addFormFont',
 '_delToC',
 '_deleteObject',
 '_delete_page',
 '_do_links',
 '_do_widgets',
 '_embeddedFileGet',
 '_embeddedFileIndex',
 '_embfile_add',
 '_embfile_del',
 '_embfile_info',
 '_embfile_names',
 '_embfile_upd',
 '_extend_toc_items',
 '_forget_page',
 '_getMetadata',
 '_getOLRootNumber',
 '_getPDFfileid',
 '_getPageInfo',
 '_get_char_widths',
 '_get_page_labels',
 '_graft_id',
 '_insert_font',
 '_loadOutline',
 '_make_page_map',
 '_m